In [1]:
%load_ext line_profiler

In [2]:
%%writefile simulation_example.py
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

def step(*shape):
    return 2 * (np.random.random_sample(shape)<.5) - 1

def simulate(iterations, n=10000):
    s = step(iterations, n)
    x = np.cumsum(s, axis=0)
    bins = np.arange(-30, 30, 1)
    y = np.vstack([np.histogram(x[i,:], bins)[0]
                   for i in range(iterations)])
    return y

Overwriting simulation_example.py


In [3]:
from simulation_example import simulate
%lprun -T lprof0_gcn.txt -f simulate simulate(200)
print(open('lprof0_gcn.txt', 'r').read())


*** Profile printout saved to text file 'lprof0_example.txt'. 
Timer unit: 1e-06 s

Total time: 0.016201 s
File: /home/xiangli/projects/pytorch_tutorial/simulation_example.py
Function: simulate at line 29

Line #      Hits         Time  Per Hit   % Time  Line Contents
    29                                           def simulate(iterations, n=10000):
    30         1       4880.0   4880.0     30.1      s = step(iterations, n)
    31         1       1411.0   1411.0      8.7      x = np.cumsum(s, axis=0)
    32         1          7.0      7.0      0.0      bins = np.arange(-30, 30, 1)
    33         1          1.0      1.0      0.0      y = np.vstack([np.histogram(x[i,:], bins)[0]
    34         1       9901.0   9901.0     61.1                     for i in range(iterations)])
    35         1          1.0      1.0      0.0      return y
